In [2]:
import pandas as pd

# File path
file_path =r'path_to_dataset
# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to verify the data is loaded correctly
print(data.head())



                                               title  \
0  Junior Data Scientist / Junior Software Developer   
1                Sr Data Scientist - Retail Pharmacy   
2                                 Sr. Data Scientist   
3           Data Scientist, Medicare Stars Analytics   
4  Senior Data Scientist (Center for Securing the...   

                                   company                location  \
0                            SynergisticIT              Boston, MA   
1                               CVS Health           Wellesley, MA   
2                          Thomson Reuters              Boston, MA   
3  Blue Cross Blue Shield of Massachusetts              Boston, MA   
4                    The MITRE Corporation  Bedford, MA (+1 other)   

                                         description  salary   posted_at  \
0  2024 is almost over and we hope the Job market...   86.32  02-02-2025   
1  Bring your heart to CVS Health. Every one of u...    8.19  30-01-2025   
2  Location: H

In [3]:
# Convert `posted_at` to datetime format
data['posted_at'] = pd.to_datetime(data['posted_at'], format='%d-%m-%Y')

# Extract year and month for trend analysis
data['year'] = data['posted_at'].dt.year
data['month'] = data['posted_at'].dt.month

# Check the result
data[['posted_at', 'year', 'month']].head()


,posted_at,year,month
0,2025-02-02,2025,2
1,2025-01-30,2025,1
2,2025-01-11,2025,1
3,2025-01-06,2025,1
4,2025-01-28,2025,1


In [4]:
import ast
from collections import Counter

# Convert the stringified list in `skills` column to an actual list
data['skills'] = data['skills'].apply(ast.literal_eval)

# Flatten all skills into a single list for frequency analysis
all_skills = [skill for skill_list in data['skills'] for skill in skill_list]
skill_counts = Counter(all_skills)

# Display the top 10 most common skills
skill_counts.most_common(10)


[('AWS', 148),
 ('Python', 147),
 ('Agile', 128),
 ('SQL', 115),
 ('Machine Learning', 95),
 ('Java', 86),
 ('C++', 75),
 ('JavaScript', 69),
 ('Azure', 67),
 ('Cybersecurity', 66)]

In [5]:
# Normalize salary (Min-Max Scaling)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data['salary_normalized'] = scaler.fit_transform(data[['salary']])

# Bin salaries into ranges
bins = [0, 50, 100, 150, 200, 500]
labels = ['0-50k', '50-100k', '100-150k', '150-200k', '200k+']
data['salary_bins'] = pd.cut(data['salary'], bins=bins, labels=labels)

# Check the result
data[['salary', 'salary_normalized', 'salary_bins']].head()


,salary,salary_normalized,salary_bins
0,86.32,0.156415,50-100k
1,8.19,0.011138,0-50k
2,45.00,0.079583,0-50k
3,24.00,0.040536,0-50k
4,45.00,0.079583,0-50k


In [6]:
# Simplify multi-location entries (e.g., "Boston, MA (+1 other)")
data['location'] = data['location'].str.split('(').str[0].str.strip()

# Extract city and state separately if needed
data[['city', 'state']] = data['location'].str.split(',', expand=True)

# Check the result
data[['location', 'city', 'state']].head()


,location,city,state
0,"Boston, MA",Boston,MA
1,"Wellesley, MA",Wellesley,MA
2,"Boston, MA",Boston,MA
3,"Boston, MA",Boston,MA
4,"Bedford, MA",Bedford,MA


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to extract top keywords
def extract_top_keywords(text_column, top_n=10):
    vectorizer = CountVectorizer(stop_words='english', max_features=top_n)
    matrix = vectorizer.fit_transform(data[text_column])
    keywords = vectorizer.get_feature_names_out()
    return keywords

# Extract keywords from `description` and `qualifications`
description_keywords = extract_top_keywords('description')
qualification_keywords = extract_top_keywords('qualifications')

# Print top keywords
print("Description Keywords:", description_keywords)
print("Qualification Keywords:", qualification_keywords)


Description Keywords: ['business' 'data' 'experience' 'management' 'security' 'skills' 'support'
 'team' 'technical' 'work']
Qualification Keywords: ['ability' 'data' 'degree' 'experience' 'knowledge' 'management' 'skills'
 'strong' 'work' 'years']


In [8]:
# Group by year and month for trend analysis
job_trends = data.groupby(['year', 'month']).size().reset_index(name='job_count')

# Check the result
job_trends.head()


,year,month,job_count
0,2025,1,373
1,2025,2,68


In [9]:
# Calculate salary IQR (Interquartile Range)
Q1 = data['salary'].quantile(0.25)
Q3 = data['salary'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
data_cleaned = data[(data['salary'] >= lower_bound) & (data['salary'] <= upper_bound)]

# Check the result
data_cleaned['salary'].describe()


count    410.000000
mean      52.406341
std       50.743109
min        2.200000
25%       10.000000
50%       31.125000
75%       88.706250
max      228.250000
Name: salary, dtype: float64

In [10]:
# Define custom job categories
def classify_job(title):
    if 'Data Scientist' in title:
        return 'Data Scientist'
    elif 'Analyst' in title:
        return 'Analyst'
    elif 'Engineer' in title:
        return 'Engineer'
    else:
        return 'Other'

# Apply classification
data['job_category'] = data['title'].apply(classify_job)

# Check the result
data[['title', 'job_category']].head()


,title,job_category
0,Junior Data Scientist / Junior Software Developer,Data Scientist
1,Sr Data Scientist - Retail Pharmacy,Data Scientist
2,Sr. Data Scientist,Data Scientist
3,"Data Scientist, Medicare Stars Analytics",Data Scientist
4,Senior Data Scientist (Center for Securing the...,Data Scientist


In [11]:
# Encode categorical columns
data['company_encoded'] = data['company'].astype('category').cat.codes
data['location_encoded'] = data['location'].astype('category').cat.codes

# Check the result
data[['company', 'company_encoded', 'location', 'location_encoded']].head()


,company,company_encoded,location,location_encoded
0,SynergisticIT,284,"Boston, MA",23
1,CVS Health,49,"Wellesley, MA",208
2,Thomson Reuters,309,"Boston, MA",23
3,Blue Cross Blue Shield of Massachusetts,37,"Boston, MA",23
4,The MITRE Corporation,307,"Bedford, MA",14


In [12]:
# Save the cleaned dataset for Tableau visualization
data.to_csv('processed_jobs_data.csv', index=False)
print("Processed dataset saved as 'processed_jobs_data.csv'")


Processed dataset saved as 'processed_jobs_data.csv'
